**Sentence Ordering**

In [1]:
import numpy as np
import pandas as pd
import os
import json
import nltk.data
import random
import re
import sklearn
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Download and using the NLTK Punkt Tokenizer

nltk.download('punkt')
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
# 5 different Permutations of the Summary and capping the number of sentences considerd to 10

SHUFFLE_COUNT = 5
MAX_SENTENCE_LENGTH = 10

In [4]:
# Preprocessing Data

os.makedirs("./arxiv_data/original/", exist_ok=True)
os.makedirs("./arxiv_data/permutations/", exist_ok=True)
os.makedirs("./arxiv_data/processed/", exist_ok=True)

In [5]:
# Reading ARXIV Data

with open("./arxivData.json", "r") as f:
    data = json.load(f)

In [6]:
# Data Preprocessing


for index,summary in enumerate(data):    
    
    # Creating the file to store the correct ordering
    file_name = "./arxiv_data/original/original_" + str(index) + ".csv"
    f = open('%s' % file_name, 'w+')
    
    # Text Cleaning
    abstract = " ".join(summary['summary'].strip())
    re.sub('[^A-Za-z0-9]+', '', abstract)
    tokenized = tokenizer.tokenize(abstract)

    processed_sent = list(map(lambda st: str.replace(st, "   ", "<><>"), tokenized))
    tokenized = list(map(lambda st: str.replace(st, " ", ""), processed_sent))
    tokenized = list(map(lambda st: str.replace(st, "<><>", " "), tokenized))

    for line_no,sent in enumerate(tokenized):
        f.write(str(line_no) + ">> " + sent + "\n")
    
    f.close()

In [7]:
# Generating Permutations

for index,summary in enumerate(data):
    
    file_name = "./arxiv_data/original/original_" + str(index) + ".csv"
    f = open('%s' % file_name, 'r')
    processed_data = f.readlines()
    
    for i in range(SHUFFLE_COUNT):
        
        perm_fname = "./arxiv_data/permutations/original_" + str(index) +"_permutation_" + str(i) + ".csv"
        sf = open('%s' % perm_fname, 'w+')
        
        if(i == 0):
            shuffled =''.join(processed_data)
        else:
            random.shuffle(processed_data)
            shuffled = ''.join(processed_data)
            
        sf.write(shuffled)
        sf.close()
    
    f.close()

In [8]:
# Read the generated permutations

permutation_files =  os.listdir("./arxiv_data/permutations/")
permutations = []

for f in permutation_files:
    
    file = open("./arxiv_data/permutations/" + f)
    permutations.append(file.readlines())

In [9]:
# Storing the original and permuted information [similar to one hot encoding]

org = np.array([[" " for i in range(MAX_SENTENCE_LENGTH)] for i in range(len(permutations))], 
                         dtype = object)
perm = np.array([[-1 for i in range(MAX_SENTENCE_LENGTH)] for i in range(len(permutations))])

for i in range(len(permutations)):
    
    info = permutations[i]
    j = 0

    for line in info:
        org[i][j] = ''.join(line.split('>> ')[1:]).strip()

        try:
            if(int(line.split('>> ')[0]) <= MAX_SENTENCE_LENGTH):
                perm[i][j] = int(line.split('>> ')[0])
                j += 1
            if( j >= MAX_SENTENCE_LENGTH ):
                break

        except (RuntimeError, TypeError, NameError, ValueError):
            pass

In [10]:
# Storing Sentece information

with open('./arxiv_data/processed/sentences.txt','w+') as f:
    for line in org:
        f.write('\t'.join(line).strip() + '\n')

In [11]:
# Storing Permuation information

with open("./arxiv_data/processed/permutations.txt",'w+') as f:
    for line in perm:
        f.write('\t'.join(line.astype(str)).strip() + '\n')

In [12]:
data_folder = './arxiv_data/processed/'
modelFileName = 'model.h5'
inc = 1
split_at = 1800
batch_size = 10
max_seq_len = 10
max_sent_len = 40
max_num_word = 2000
EMBEDDING_DIM = 300
useWordLevelEmbeddings = True
usePretrainedWordEmbeddings = False

In [13]:
y = np.loadtxt(data_folder + 'permutations.txt', delimiter='\t', dtype=int)

with open(data_folder + 'sentences.txt', encoding='utf8') as f:
    sentences = f.readlines()

YY = []
for y_ in y:
    yy = []
    dummyVec = np.array([ 0 for i in range(MAX_SENTENCE_LENGTH + inc)])
    for yy_ in y_:
        dummyVec[yy_] = 1
        yy.append(np.copy(dummyVec))
        dummyVec[yy_] = 0
    YY.append(yy)

YY = np.asarray(YY)

tokenizer_file_extra = ''

X = []
for line in sentences:
    lineSents = line.split('\t')
    xx = [[" "] for i in range(MAX_SENTENCE_LENGTH)]
    j = 0
    for ls in lineSents:
        xx[j][0] = ls
        j += 1
    X.append(xx)

X = np.asarray(X)
    
# split into train and test data
x_train = X[:split_at]
x_test = X[split_at:]

y_train = YY[:split_at]
y_test = YY[split_at:]

validation_data = (x_test, y_test)

In [14]:
y_test[0]

array([[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

In [15]:
x_test[0]

array([['The architecture shows'],
       ['This relationship is exploited to'],
       ['In this paper we present a novel Neural Network algorithm for conducting'],
       ['We introduce a neural\n'],
       [' '],
       [' '],
       [' '],
       [' '],
       [' '],
       [' ']], dtype='<U80')

In [16]:
import pandas as pd
import numpy as np
import os
import sys
from keras.layers import Dense, Dropout
from keras.models import Model
from keras.models import model_from_json
from keras.models import Sequential
from keras.layers import Dense,Input
from keras.utils import to_categorical
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation,Lambda
import json
import pickle

In [17]:
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
import tensorflow_hub as hub

In [ ]:
x_final_train = []
for para in x_train:
	sent = []
	for s in para:
		sent.append(s[0])

	embeddings = elmo_model(sent, signature="default",as_dict=True)["elmo"]

	mean_embedding = tf.math.reduce_mean(embeddings_1, axis=1, keepdims=False, name=None)

	x_final_train.append(mean_embedding)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [80]:
embeddings_1 = elmo_model(
    ["the cat is on the", "hat on mat"],
    signature="default",
    as_dict=True)["elmo"]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [81]:
embeddings_1

<tf.Tensor 'elmo_module_1_apply_default_7/aggregation/mul_3:0' shape=(2, 5, 1024) dtype=float32>

In [82]:
mean_embedding = tf.math.reduce_mean(
    embeddings_1, axis=1, keepdims=False, name=None
)

In [83]:
mean_embedding

<tf.Tensor 'Mean_5:0' shape=(2, 1024) dtype=float32>

In [19]:
# Elmo Model

elmo_model = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True, 
name='elmo_module')

In [93]:
embeddings = elmo_model(
    ["the cat is on the mat", "dogs are in the fog", "me hear"],
    signature="default",
    as_dict=True)["elmo"]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [62]:
embeddings

<tf.Tensor 'elmo_module_1_apply_default_2/aggregation/mul_3:0' shape=(1800, 108, 1024) dtype=float32>

In [63]:
embeddings[0]

<tf.Tensor 'strided_slice_6:0' shape=(108, 1024) dtype=float32>

In [64]:
embeddings.get_shape

<bound method Tensor.get_shape of <tf.Tensor 'elmo_module_1_apply_default_2/aggregation/mul_3:0' shape=(1800, 108, 1024) dtype=float32>>

In [68]:
mean_embedding[0]

<tf.Tensor 'strided_slice_8:0' shape=(1024,) dtype=float32>

In [22]:
x_train[0]

array([['Experience replay is a key technique behind many recent advances in deep'],
       ['Allowing the agent to learn from earlier memories can'],
       ['Despite its'],
       ['How does the amount of memory kept affect learning dynamics?'],
       ['Does it help to prioritize certain experiences?'],
       ['In this paper, we address these'],
       ['We derive analytic solutions of the ODE for a simple'],
       ['We show that even in this very simple setting, the amount of memory'],
       ['Too much or too little'],
       ['Moreover, we characterize regimes where\n']], dtype='<U80')

In [23]:
y_train[0]

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]])

In [48]:
x_train[0]

array([['Experience replay is a key technique behind many recent advances in deep'],
       ['Allowing the agent to learn from earlier memories can'],
       ['Despite its'],
       ['How does the amount of memory kept affect learning dynamics?'],
       ['Does it help to prioritize certain experiences?'],
       ['In this paper, we address these'],
       ['We derive analytic solutions of the ODE for a simple'],
       ['We show that even in this very simple setting, the amount of memory'],
       ['Too much or too little'],
       ['Moreover, we characterize regimes where\n']], dtype='<U80')

In [49]:
embeddings[0]

<tf.Tensor 'strided_slice_4:0' shape=(19, 1024) dtype=float32>

In [24]:
x_vectorized_comb  = np.concatenate((x_train.flatten(), x_test.flatten()), axis=None)
x_vectorized_train = x_train.flatten()
x_vectorized_test  = x_train.flatten()

In [25]:
embeddings = elmo_model(
    x_vectorized_comb,
    signature="default",
    as_dict=True)["elmo"]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [47]:
embeddings.get_shape

<bound method Tensor.get_shape of <tf.Tensor 'elmo_module_1_apply_default_1/aggregation/mul_3:0' shape=(2050000, 19, 1024) dtype=float32>>

In [28]:
type(embeddings)

tensorflow.python.framework.ops.Tensor

In [29]:
embeddings.get_shape

<bound method Tensor.get_shape of <tf.Tensor 'elmo_module_1_apply_default_1/aggregation/mul_3:0' shape=(2050000, 19, 1024) dtype=float32>>

In [35]:
feature_columns = [tf.feature_column.numeric_column('x', shape=x_train.shape[1:])]

In [36]:
feature_columns

[NumericColumn(key='x', shape=(10, 1), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [44]:
feature_columns = [tf.feature_column.numeric_column('x', shape=x_train.shape[1:])]
len(feature_columns)

1

In [31]:
def BiLstmBased(max_num_sent):
    sent_seq = Input(shape=(max_num_sent, 1), name='sent_inp', dtype=tf.string)
    # sent_emb = TimeDistributed(Lambda(UniversalEmbedding, output_shape=(512,)))(sent_seq)
    sent_lstm = Bidirectional(LSTM(100, return_sequences=True))(sent_seq)
    sent_lstm = Dropout(0.5)(sent_lstm)
    final_lstm = Bidirectional(LSTM(300, return_sequences=True))(sent_lstm)
    final_lstm = Dropout(0.5)(final_lstm)
    output_seq = TimeDistributed(Dense(max_num_sent + 1, activation='softmax'))(final_lstm) 
    model = Model(inputs = sent_seq, outputs=output_seq)
    model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])
    return model


In [ ]:
from keras.layers import Dense # Dense layers are "fully connected" layers
from keras.models import Sequential # Documentation: https://keras.io/models/sequential/

image_size = 784 # 28*28
num_classes = 10 # ten unique digits

model = Sequential()

# The input layer requires the special input_shape parameter which should match
# the shape of our training data.
model.add(Dense(units=32, activation='sigmoid', input_shape=(image_size,)))
model.add(Dense(units=num_classes, activation='softmax'))
model.summary()


ModuleNotFoundError: ignored

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from keras.layers import Input, Dense, LSTM, Lambda, TimeDistributed, Bidirectional, Dropout, BatchNormalization
from keras import Model
import keras.backend as K

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
data_folder = './arxiv_data/processed/'
modelFileName = 'model.h5'
split_at = 1800
batch_size = 10
max_seq_len = 10
max_sent_len = 40
max_num_word = 2000
EMBEDDING_DIM = 300
useWordLevelEmbeddings = True
usePretrainedWordEmbeddings = False

In [ ]:
def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)

In [ ]:
# module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"
# # Import the Universal Sentence Encoder's TF Hub module
# embed = hub.load(module_url)

# # embed = hub.load('https://tfhub.dev/google/nnlm-en-dim128/1')
# # embed.signatures['default'](['my text', 'batch'])

In [ ]:
embed = hub.load('https://tfhub.dev/google/nnlm-en-dim128/1')
embed.signatures['default']

<ConcreteFunction pruned(default) at 0x7FB3237579E8>

In [ ]:
def BiLstmBased(max_num_sent):
    sent_seq = Input(shape=(max_num_sent, 1), name='sent_inp', dtype=tf.string)
    sent_emb = TimeDistributed(Lambda(UniversalEmbedding, output_shape=(512,)))(sent_seq)
    sent_lstm = Bidirectional(LSTM(100, return_sequences=True))(sent_emb)
    sent_lstm = Dropout(0.5)(sent_lstm)
    final_lstm = Bidirectional(LSTM(300, return_sequences=True))(sent_lstm)
    final_lstm = Dropout(0.5)(final_lstm)
    output_seq = TimeDistributed(Dense(max_num_sent + 1, activation='softmax'))(final_lstm) 
    model = Model(inputs = sent_seq, outputs=output_seq)
    model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])
    return model

In [ ]:
# Shreya

from tensorflow.compat.v1.keras import backend as K

model = BiLstmBased(max_seq_len)
 
os.makedirs("./Models/", exist_ok=True)
with tf.Session() as session:
  tf.compat.v1.keras.backend.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  K.set_session(session)
  history = model.fit(x_train, y_train, epochs=10, batch_size=batch_size,
                    validation_data=validation_data, shuffle=True)

  model.save_weights('./Models/' + modelFileName )

TypeError: ignored